In [1]:
import uproot
from impy.models import Pythia8
from impy.kinematics import CenterOfMass
from impy.constants import GeV, quarks_and_diquarks_and_gluons
import awkward as ak
import numpy as np
from particle import literals as lp

In [2]:
model = Pythia8(CenterOfMass(10 * GeV, "proton", "proton"), seed=1)
model.set_stable(lp.pi_0.pdgid, False)


 *------------------------------------------------------------------------------------* 
 |                                                                                    | 
 |  *------------------------------------------------------------------------------*  | 
 |  |                                                                              |  | 
 |  |                                                                              |  | 
 |  |   PPP   Y   Y  TTTTT  H   H  III    A      Welcome to the Lund Monte Carlo!  |  | 
 |  |   P  P   Y Y     T    H   H   I    A A     This is PYTHIA version 8.307      |  | 
 |  |   PPP     Y      T    HHHHH   I   AAAAA    Last date of change: 25 Feb 2022  |  | 
 |  |   P       Y      T    H   H   I   A   A                                      |  | 
 |  |   P       Y      T    H   H  III  A   A    Now is 18 Oct 2022 at 18:25:21    |  | 
 |  |                                                                              |  | 
 |  |   Program docu

In [6]:
nevents = 1000

def write(file, lengths, px, py, pz, m, vx, vy, vz, pid, status, par):
    b = np.sum(lengths)
    data = {
        "": ak.zip({
            "px": ak.unflatten(px[:b], lengths),
            "py": ak.unflatten(py[:b], lengths),
            "pz": ak.unflatten(pz[:b], lengths),
            "m": ak.unflatten(m[:b], lengths),
            "vx": ak.unflatten(vx[:b], lengths),
            "vy": ak.unflatten(vy[:b], lengths),
            "vz": ak.unflatten(vz[:b], lengths),
            "pid": ak.unflatten(pid[:b], lengths),
            "status": ak.unflatten(status[:b], lengths),
            "par": ak.unflatten(par[:b], lengths),
        }),
    }
    if "tree" in file:
        file["tree"].extend(data)
    else:
        file["tree"] = data

with uproot.recreate("output.root") as f:
    px = np.empty(100000)
    py = np.empty(100000)
    pz = np.empty(100000)
    m = np.empty(100000)
    vx = np.empty(100000)
    vy = np.empty(100000)
    vz = np.empty(100000)
    pid = np.empty(100000, np.int32)
    status = np.empty(100000, np.int32)
    par = np.empty(100000, np.int32)
    lengths = []
    b = 0
    for event in model(nevents):
        mask = True
        apid = np.abs(event.pid)
        for pdg in quarks_and_diquarks_and_gluons:
            mask &= apid != pdg
        mask[:2] = False
        out = event[mask]
        a = b
        b += len(out)
        lengths.append(b - a)
        px[a:b] = out.px
        py[a:b] = out.py
        pz[a:b] = out.pz
        m[a:b] = out.m
        vx[a:b] = out.vx
        vy[a:b] = out.vy
        vz[a:b] = out.vz
        pid[a:b] = out.pid
        status[a:b] = out.status
        par[a:b] = out.parents[:, 0] - 1
        if b > 50000:
            write(f, lengths, px, py, pz, m, vx, vy, vz, pid, status, par)
            lengths = []
            b = 0
    # write the rest
    if b:
        write(f, lengths, px, py, pz, m, vx, vy, vz, pid, status, par)


 Pythia::next(): 10000 events have been generated 

 Pythia::next(): 11000 events have been generated 

 Pythia::next(): 12000 events have been generated 

 Pythia::next(): 13000 events have been generated 

 Pythia::next(): 14000 events have been generated 

 Pythia::next(): 15000 events have been generated 

 Pythia::next(): 16000 events have been generated 

 Pythia::next(): 17000 events have been generated 

 Pythia::next(): 18000 events have been generated 

 Pythia::next(): 19000 events have been generated 


In [26]:
import numpy as np
import uproot
from particle import literals as lp
import numba as nb


def pt_eta(px, py, pz):
    pt = np.sqrt(px ** 2 + py ** 2)
    eta = np.arcsinh(pz / pt)
    return pt, eta

photon_pid = int(lp.photon.pdgid)
pi0_pid = int(lp.pi_0.pdgid)


@nb.njit
def select_photons_from_pi0_decay(pid, par):
    result = np.zeros(len(pid), dtype="bool")
    for i, pidi in enumerate(pid):
        if pidi != photon_pid:
            continue
        if par[i] < 0:
            continue
        parent_pid = pid[par[i]]
        if parent_pid != pi0_pid:
            continue
        result[i] = True
    return result

with uproot.open("output.root") as f:
    tree = f["tree"]
    branches = ["px", "py", "pz", "m", "pid", "status", "par"]
    # loop over chunks in ROOT tree
    for chunk in tree.iterate(branches):
        # loop over individual events
        for px, py, pz, m, pid, sta, par in zip(*(chunk[k] for k in branches)):
            pt, eta = pt_eta(px, py, pz)

            print("all particles")
            print("  pseudorapidity", eta)
            print("  pdgid", pid)

            # select only long-lived particles
            mask = sta == 1
            print("long-lived particles")
            print("  pseudorapidity", eta[mask])
            print("  pdgid", pid[mask])

            # select only photons from pi0 decays
            mask = select_photons_from_pi0_decay(pid, par)
            print("  photons from pi0 decay")
            print("  pseudorapidity", eta[mask])

            break  # remove to not stop after first event


all particles
  pseudorapidity [11.2, -11.2, 30.8, -0.626, 6.7, 7.32, 3.79, ... 1.61, 1.68, 4.69, 4.4, 4.76, 4.51]
  pdgid [9902210, 2212, 2212, 990, 113, 211, -211, 213, ... 22, 22, 22, 22, 22, 22, 22, 22]
long-lived particles
  pseudorapidity [-11.2, 7.32, 3.79, 3.78, 3.95, 6.74, 6.39, ... 1.61, 1.68, 4.69, 4.4, 4.76, 4.51]
  pdgid [2212, 211, -211, -211, -2112, -211, -211, 2212, ... 22, 22, 22, 22, 22, 22, 22, 22]
  photons from pi0
  pseudorapidity [3.06, 3.51, 1.61, 1.68, 4.69, 4.4, 4.76, 4.51]
